In [ ]:
# Imports
import numpy as np
import pandas as pd
import os

from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR, SVR
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.linear_model import Ridge, RidgeCV

import lightgbm as lgb
import xgboost as xgb
import time
import datetime
pd.options.display.precision = 15

import psutil
import gc
from catboost import CatBoostRegressor
import seaborn as sns
from scipy import stats

import matplotlib.pyplot as plt
get_ipython().run_line_magic('matplotlib', 'inline')
from tqdm import tqdm_notebook

from tsfresh.feature_extraction import feature_calculators

import warnings
warnings.filterwarnings('ignore')
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy as sc
import matplotlib.pyplot as plt
import seaborn as sns
import gc
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
from tqdm import tqdm_notebook
import datetime
import time
import random
from joblib import Parallel, delayed


import lightgbm as lgb
import xgboost as xgb
from tensorflow import keras
from gplearn.genetic import SymbolicRegressor
from catboost import Pool, CatBoostRegressor

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV, KFold, RandomizedSearchCV
from sklearn.feature_selection import RFECV, SelectFromModel

from sklearn.linear_model import LinearRegression, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import NuSVR, SVR
from sklearn.kernel_ridge import KernelRidge
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor

In [ ]:
print(os.listdir('../input'))

In [ ]:
# # Read the csv file to take input into train dataframe using pandas, 
# # 2 attributes from the training data: acoustic data: records the seismic activity and
# # time_to_failure: time left for the next laboratory earthquake
# train_dataset = pd.read_csv('../input/train.csv', dtype={'acoustic_data': np.int16, 'time_to_failure': np.float32})

In [ ]:
# print(train_dataset.head())

In [ ]:
# # number of instances for each segment to be specified as 150000 because each test segment has 150000 observations.
# instances = 150000
# segments = int(np.floor(train_dataset.shape[0] / instances))

# # X_trainset is the training data: acoustic data
# # y_trainset is the value to be predicted, that is the time left for the next lab earthquake
# X_trainset = pd.DataFrame(index=range(segments), dtype=np.float64)
# y_trainset = pd.DataFrame(index=range(segments), dtype=np.float64,
#                        columns=['time_to_failure'])

# # for every segment id, do feature engineering
# for segment in tqdm_notebook(range(segments)):

# #   creating segments of size 150000 starting from the row of segment id and upto segment id + instances
#     each_seg = train_dataset.iloc[segment * instances : segment * instances + instances]
# #     create x and y having acoustic_data and time_to _failure respectively
#     x_rawdata = each_seg['acoustic_data']
#     x = x_rawdata.values
#     x = pd.Series(x)
    
#     zc = np.fft.fft(x)
#     realFFT = pd.Series(np.real(zc))
#     imagFFT = pd.Series(np.imag(zc))
    
#     # most important features 30
#     X_trainset.loc[segment, 'autocorrelation_5'] = feature_calculators.autocorrelation(x, 5)
#     x_imagFFT_std = imagFFT.rolling(50).std().dropna().values
#     X_trainset.loc[segment, 'quantile_50_percentile_imagFFT_standard_deviation_50'] = np.quantile(x_imagFFT_std,0.50)
#     x_roll_std = x.rolling(1000).std().dropna().values
#     X_trainset.loc[segment, 'quantile_5_percentile_rolling_standard_deviation_1000'] = np.quantile(x_roll_std,0.05)
#     x_imagFFT_std = imagFFT.rolling(1000).std().dropna().values
#     X_trainset.loc[segment, 'quantile_50_percentile_imagFFT_standard_deviation_1000'] = np.quantile(x_imagFFT_std,0.50)
#     x_roll_std = x.rolling(50).std().dropna().values
#     X_trainset.loc[segment, 'quantile_5_percentile_rolling_standard_deviation_50'] = np.quantile(x_roll_std,0.05)
#     x_imagFFT_mean = imagFFT.rolling(100).mean().dropna().values
#     X_trainset.loc[segment, 'quantile_50_percentile_imagFFT_mean_100'] = np.quantile(x_imagFFT_mean,0.50)
#     x_roll_std = x.rolling(100).std().dropna().values
#     X_trainset.loc[segment, 'minimum_rolling_standard_deviation_100'] = x_roll_std.min()
#     x_roll_std = x.rolling(50).std().dropna().values
#     X_trainset.loc[segment, 'minimum_rolling_standard_deviation_50'] = x_roll_std.min()
#     x_roll_std = x.rolling(500).std().dropna().values
#     X_trainset.loc[segment, 'minimum_rolling_standard_deviation_500'] = x_roll_std.min()
#     x_imagFFT_std = imagFFT.rolling(100).std().dropna().values
#     X_trainset.loc[segment, 'quantile_50_percentile_imagFFT_standard_deviation_100'] = np.quantile(x_imagFFT_std,0.50)
#     x_imagFFT_std = imagFFT.rolling(10000).std().dropna().values
#     X_trainset.loc[segment, 'average_change_absolute_imagFFT_standard_deviation_10000'] = np.mean(np.diff(x_imagFFT_std))
#     x_imagFFT_std = imagFFT.rolling(50).std().dropna().values
#     X_trainset.loc[segment, 'average_change_absolute_imagFFT_standard_deviation_50'] = np.mean(np.diff(x_imagFFT_std))
#     x_realFFT_std = realFFT.rolling(500).std().dropna().values
#     X_trainset.loc[segment, 'minimum_realFFT_standard_deviation_500'] = x_realFFT_std.min()
#     x_realFFT_std = realFFT.rolling(10).std().dropna().values
#     X_trainset.loc[segment, 'quantile_5_percentile_realFFT_standard_deviation_10'] = np.quantile(x_realFFT_std,0.05)
#     x_imagFFT_mean = imagFFT.rolling(1000).mean().dropna().values
#     X_trainset.loc[segment, 'quantile_50_percentile_imagFFT_mean_1000'] = np.quantile(x_imagFFT_mean,0.50)
#     x_realFFT_std = realFFT.rolling(10).std().dropna().values
#     X_trainset.loc[segment, 'quantile_1_percentile_realFFT_standard_deviation_10'] = np.quantile(x_realFFT_std,0.01)
#     x_imagFFT_mean = imagFFT.rolling(50).mean().dropna().values
#     X_trainset.loc[segment, 'minimum_imagFFT_standard_deviation_50'] = x_imagFFT_std.min()
#     x_realFFT_mean = realFFT.rolling(1000).mean().dropna().values
#     X_trainset.loc[segment, 'quantile_5_percentile_realFFT_mean_1000'] = np.quantile(x_realFFT_mean,0.05)
#     x_realFFT_mean = realFFT.rolling(10).mean().dropna().values
#     X_trainset.loc[segment, 'maximum_realFFT_mean_10'] = x_realFFT_mean.max()
#     x_imagFFT_std = imagFFT.rolling(10).std().dropna().values
#     X_trainset.loc[segment, 'average_change_absolute_imagFFT_standard_deviation_10'] = np.mean(np.diff(x_imagFFT_std))
#     X_trainset.loc[segment, 'Isum'] = imagFFT.sum()
#     X_trainset.loc[segment, 'Rmedian_absolute_last_5000'] = np.median(np.abs(realFFT[-5000:])) 
#     X_trainset.loc[segment, 'num_peaks_10'] = feature_calculators.number_peaks(x, 10)
    
#     y = each_seg['time_to_failure'].values[-1]
    
# #   y train data is the time_to_failure for that segment instance
#     y_trainset.loc[segment, 'time_to_failure'] = y
    

In [ ]:
# submission = pd.read_csv('../input/sample_submission.csv', index_col='seg_id')
# X_testset = pd.DataFrame(columns=X_trainset.columns, dtype=np.float64, index=submission.index)

# for i, seg_id in enumerate(tqdm_notebook(X_testset.index)):
#     each_seg = pd.read_csv('../input/test/' + seg_id + '.csv')
#     x_rawdata = each_seg['acoustic_data']
#     x = x_rawdata.values
#     x = pd.Series(x)
    
#     zc = np.fft.fft(x)
#     realFFT = pd.Series(np.real(zc))
#     imagFFT = pd.Series(np.imag(zc))
    
#     # most important features 30
#     X_testset.loc[seg_id, 'autocorrelation_5'] = feature_calculators.autocorrelation(x, 5)
#     x_imagFFT_std = imagFFT.rolling(50).std().dropna().values
#     X_testset.loc[seg_id, 'quantile_50_percentile_imagFFT_standard_deviation_50'] = np.quantile(x_imagFFT_std,0.50)
#     x_roll_std = x.rolling(1000).std().dropna().values
#     X_testset.loc[seg_id, 'quantile_5_percentile_rolling_standard_deviation_1000'] = np.quantile(x_roll_std,0.05)
#     x_imagFFT_std = imagFFT.rolling(1000).std().dropna().values
#     X_testset.loc[seg_id, 'quantile_50_percentile_imagFFT_standard_deviation_1000'] = np.quantile(x_imagFFT_std,0.50)
#     x_roll_std = x.rolling(50).std().dropna().values
#     X_testset.loc[seg_id, 'quantile_5_percentile_rolling_standard_deviation_50'] = np.quantile(x_roll_std,0.05)
#     x_imagFFT_mean = imagFFT.rolling(100).mean().dropna().values
#     X_testset.loc[seg_id, 'quantile_50_percentile_imagFFT_mean_100'] = np.quantile(x_imagFFT_mean,0.50)
#     x_roll_std = x.rolling(100).std().dropna().values
#     X_testset.loc[seg_id, 'minimum_rolling_standard_deviation_100'] = x_roll_std.min()
#     x_roll_std = x.rolling(50).std().dropna().values
#     X_testset.loc[seg_id, 'minimum_rolling_standard_deviation_50'] = x_roll_std.min()
#     x_roll_std = x.rolling(500).std().dropna().values
#     X_testset.loc[seg_id, 'minimum_rolling_standard_deviation_500'] = x_roll_std.min()
#     x_imagFFT_std = imagFFT.rolling(100).std().dropna().values
#     X_testset.loc[seg_id, 'quantile_50_percentile_imagFFT_standard_deviation_100'] = np.quantile(x_imagFFT_std,0.50)
#     x_imagFFT_std = imagFFT.rolling(10000).std().dropna().values
#     X_testset.loc[seg_id, 'average_change_absolute_imagFFT_standard_deviation_10000'] = np.mean(np.diff(x_imagFFT_std))
#     x_imagFFT_std = imagFFT.rolling(50).std().dropna().values
#     X_testset.loc[seg_id, 'average_change_absolute_imagFFT_standard_deviation_50'] = np.mean(np.diff(x_imagFFT_std))
#     x_realFFT_std = realFFT.rolling(500).std().dropna().values
#     X_testset.loc[seg_id, 'minimum_realFFT_standard_deviation_500'] = x_realFFT_std.min()
#     x_realFFT_std = realFFT.rolling(10).std().dropna().values
#     X_testset.loc[seg_id, 'quantile_5_percentile_realFFT_standard_deviation_10'] = np.quantile(x_realFFT_std,0.05)
#     x_imagFFT_mean = imagFFT.rolling(1000).mean().dropna().values
#     X_testset.loc[seg_id, 'quantile_50_percentile_imagFFT_mean_1000'] = np.quantile(x_imagFFT_mean,0.50)
#     x_realFFT_std = realFFT.rolling(10).std().dropna().values
#     X_testset.loc[seg_id, 'quantile_1_percentile_realFFT_standard_deviation_10'] = np.quantile(x_realFFT_std,0.01)
#     x_imagFFT_mean = imagFFT.rolling(50).mean().dropna().values
#     X_testset.loc[seg_id, 'minimum_imagFFT_standard_deviation_50'] = x_imagFFT_std.min()
#     x_realFFT_mean = realFFT.rolling(1000).mean().dropna().values
#     X_testset.loc[seg_id, 'quantile_5_percentile_realFFT_mean_1000'] = np.quantile(x_realFFT_mean,0.05)
#     x_realFFT_mean = realFFT.rolling(10).mean().dropna().values
#     X_testset.loc[seg_id, 'maximum_realFFT_mean_10'] = x_realFFT_mean.max()
#     x_imagFFT_std = imagFFT.rolling(10).std().dropna().values
#     X_testset.loc[seg_id, 'average_change_absolute_imagFFT_standard_deviation_10'] = np.mean(np.diff(x_imagFFT_std))
#     X_testset.loc[seg_id, 'Isum'] = imagFFT.sum()
#     X_testset.loc[seg_id, 'Rmedian_absolute_last_5000'] = np.median(np.abs(realFFT[-5000:])) 
#     X_testset.loc[seg_id, 'num_peaks_10'] = feature_calculators.number_peaks(x, 10)

In [ ]:
# X_trainset.to_csv('X_trainset.csv')
# X_testset.to_csv('X_testset.csv')
# y_trainset.to_csv('y_trainset.csv')

In [ ]:
X_trainset = pd.read_csv('../input/earthquakefe/X_trainset.csv').reset_index(drop=True)
X_testset = pd.read_csv('../input/earthquakefe/X_testset.csv').reset_index(drop=True)
y_trainset = pd.read_csv('../input/earthquakefe/y_trainset.csv')['time_to_failure']
submission = pd.read_csv('../input/LANL-Earthquake-Prediction/sample_submission.csv', index_col='seg_id')

In [ ]:
print(y_trainset.shape)
y_trainset.head()
X_trainset.head()
del X_trainset['Unnamed: 0']
del X_testset["seg_id"]
print(X_trainset.shape)
print(X_testset.shape)

# LGBM

In [ ]:
num_folds = 5
k_folds = KFold(n_splits=num_folds, shuffle=True, random_state=630)

In [ ]:
def train_model_lgb(X=X_trainset, X_testset=X_testset, y=y_trainset, params=None, k_folds=k_folds, model=None):
    columns = X.columns
    
    x_values = np.zeros(len(X))
    prediction = np.zeros(len(X_testset))
    scores = []
    feature_importance = pd.DataFrame()
    for fold_n, (trainset_index, valid_set_index) in enumerate(k_folds.split(X)):
        print('Fold', fold_n, 'started at', time.ctime())
        X_train_per_fold, X_valid_per_fold = X.iloc[trainset_index], X.iloc[valid_set_index]
        y_train_per_fold, y_valid_per_fold = y.iloc[trainset_index], y.iloc[valid_set_index]

        model = lgb.LGBMRegressor(**params, n_estimators = 50000, nthread = 4, n_jobs = -1)
        model.fit(X_train_per_fold, y_train_per_fold, 
               eval_set=[(X_train_per_fold, y_train_per_fold), (X_valid_per_fold, y_valid_per_fold)], eval_metric='mae',
               verbose=10000, early_stopping_rounds=200)

        y_pred_valid = model.predict(X_valid_per_fold)
        y_pred = model.predict(X_testset, num_iteration=model.best_iteration_)

        x_values[valid_set_index] = y_pred_valid.reshape(-1,)
        scores.append(mean_absolute_error(y_valid_per_fold, y_pred_valid))
        
        fold_importance = pd.DataFrame()
        fold_importance["feature"] = columns
        fold_importance["importance"] = model.feature_importances_
        fold_importance["fold"] = fold_n + 1
        feature_importance = pd.concat([feature_importance, fold_importance], axis=0)
        
        prediction += y_pred
        
    prediction /= num_folds
#     feature_importance["importance"] /= k_folds.n_splits
    cols = feature_importance[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)[:30].index
    best_features = feature_importance.loc[feature_importance.feature.isin(cols)]

    plt.figure(figsize=(16, 12));
    sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False));
    plt.title('LGB Features (avg over folds)');

    print('CV mean score: {0:.4f}, std: {1:.4f}.'.format(np.mean(scores), np.std(scores)))
    return x_values, prediction

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# lgb_params = {'num_leaves': 32,
#               'min_data_in_leaf': 16,
#               'objective': 'regression',
#               'max_depth': -1,
#               'learning_rate': 0.001,
#               "boosting": "gbdt",
#               "feature_fraction": 0.8,
#               "bagging_freq": 2,
#               "bagging_fraction": 0.8,
#               "bagging_seed": 630,
#               "metric": 'mae',
#               "verbosity": -1,
#               'reg_alpha': 0.15,
#               'reg_lambda': 0.36,
#          }
lgb_params = {'num_leaves': 21,
              'min_data_in_leaf': 20,
              'objective': 'regression',
              'max_depth': 108,
              'learning_rate': 0.001,
              'boosting': 'gbdt',
              'feature_fraction': 0.8,
              'bagging_freq': 2,
              'bagging_fraction': 0.8,
              'bagging_seed': 630,
              'metric': 'mae',
              'verbosity': -1,
              'reg_alpha': 0.15,
              'reg_lambda': 0.36,
              'random_state': 630,
         }
x_value_lgb, prediction_lgb = train_model_lgb(params = lgb_params)

In [ ]:
submission = pd.read_csv('../input/LANL-Earthquake-Prediction/sample_submission.csv')
submission['time_to_failure'] = prediction_lgb
print(submission.head())
submission.to_csv('30_lgbm.csv')

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

lgb_params = {'num_leaves': 32,
              'min_data_in_leaf': 16,
              'objective': 'regression',
              'max_depth': -1,
              'learning_rate': 0.001,
              "boosting": "gbdt",
              "feature_fraction": 0.8,
              "bagging_freq": 2,
              "bagging_fraction": 0.8,
              "bagging_seed": 630,
              "metric": 'mae',
              "verbosity": -1,
              'reg_alpha': 0.15,
              'reg_lambda': 0.36,
         }
x_value_lgb, prediction_lgb_2 = train_model_lgb(params = lgb_params)

In [ ]:
submission = pd.read_csv('../input/LANL-Earthquake-Prediction/sample_submission.csv')
submission['time_to_failure'] = prediction_lgb_2
print(submission.head())
submission.to_csv('30_lgbm_2.csv')

# NN

In [ ]:
scaler = StandardScaler()
train_columns = X_trainset.columns

X_trainset[train_columns] = scaler.fit_transform(X_trainset[train_columns])
X_testset[train_columns] = scaler.transform(X_testset[train_columns])

In [ ]:
def create_model(input_dim=10):

    model = keras.Sequential()
    model.add(keras.layers.Dense(256, activation="relu", use_bias=True, kernel_initializer='he_normal', input_dim=input_dim))
    model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.Dense(128, activation="relu", use_bias=True, kernel_initializer='he_normal'))
    model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.Dense(96, activation="relu", use_bias=True, kernel_initializer='he_normal'))
    model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.Dense(1, activation="linear"))

    optimizer = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False) #'rmsprop'
    model.compile(optimizer=optimizer,loss='mae', metrics=['accuracy'])
    return model

patience = 50
call_ES = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=patience, verbose=1, mode='auto', baseline=None, restore_best_weights=True)

In [ ]:
%%time
n_fold = 5
folds = KFold(n_splits=n_fold, shuffle=True, random_state=630)

NN_oof = np.zeros(len(X_trainset))
train_score = []
fold_idxs = []

NN_predictions = np.zeros(len(X_testset))


for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_trainset,y_trainset.values)):
    strLog = "fold {}".format(fold_)
    print(strLog)
    fold_idxs.append(val_idx)
    
    X_tr, X_val = X_trainset[train_columns].iloc[trn_idx], X_trainset[train_columns].iloc[val_idx]
    y_tr, y_val = y_trainset.iloc[trn_idx], y_trainset.iloc[val_idx]
    model = create_model(X_trainset.shape[-1])
    model.fit(X_tr, y_tr, epochs=500, batch_size=32, verbose=0, callbacks=[call_ES,], validation_data=[X_val, y_val]) #
    
    NN_oof[val_idx] = model.predict(X_val)[:,0]
    
    NN_predictions += model.predict(X_testset[train_columns])[:,0] / folds.n_splits
    history = model.history.history
    tr_loss = history["loss"]
    val_loss = history["val_loss"]
    print(f"loss: {tr_loss[-patience]:.3f} | val_loss: {val_loss[-patience]:.3f} | diff: {val_loss[-patience]-tr_loss[-patience]:.3f}")
    train_score.append(tr_loss[-patience])
#     break
    
cv_score = mean_absolute_error(y_trainset, NN_oof)
print(f"After {n_fold} test_CV = {cv_score:.3f} | train_CV = {np.mean(train_score):.3f} | {cv_score-np.mean(train_score):.3f}", end=" ")

In [ ]:
submission = pd.read_csv('../input/LANL-Earthquake-Prediction/sample_submission.csv')
submission["time_to_failure"] = NN_predictions
submission.to_csv('30_nn.csv', index=False)
submission.head()

# CAT

In [ ]:
num_folds = 5
k_folds = KFold(n_splits=num_folds, shuffle=True, random_state=630)

In [ ]:
def train_model_cat(X=X_trainset, X_testset=X_testset, y=y_trainset, params=None, k_folds=k_folds, model=None):
    train_columns = X.columns
    
    x_values = np.zeros(len(X))
    prediction = np.zeros(len(X_testset))
    scores = []
    feature_importance = pd.DataFrame()

    #run model
    for fold_n, (trn_idx, val_idx) in enumerate(k_folds.split(X,y.values)):
        print('Fold', fold_n, 'started at', time.ctime())

        X_tr, X_val = X_trainset[train_columns].iloc[trn_idx], X_trainset[train_columns].iloc[val_idx]
        y_tr, y_val = y_trainset.iloc[trn_idx], y_trainset.iloc[val_idx]

        model = CatBoostRegressor(n_estimators=25000, eval_metric='MAE',
                                  random_seed=630, l2_leaf_reg=3,
                                  verbose=-1, objective="MAE", loss_function="MAE", 
                                  boosting_type="Ordered")
        model.fit(X_tr, 
                  y_tr, 
                  eval_set=[(X_val, y_val)], 
                  verbose=2500, 
                  early_stopping_rounds=250)
        x_values[val_idx] = model.predict(X_val)
        
        y_pred_valid = model.predict(X_val)
        y_pred = model.predict(X_testset)

        x_values[val_idx] = y_pred_valid.reshape(-1,)
        scores.append(mean_absolute_error(y_val, y_pred_valid))
        
        fold_importance = pd.DataFrame()
        fold_importance["feature"] = train_columns
        fold_importance["importance"] = model.feature_importances_
        fold_importance["fold"] = fold_n + 1
        feature_importance = pd.concat([feature_importance, fold_importance], axis=0)
        
        prediction += y_pred
        
    prediction /= num_folds
    cols = feature_importance[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)[:30].index
    best_features = feature_importance.loc[feature_importance.feature.isin(cols)]

    plt.figure(figsize=(16, 12));
    sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False));
    plt.title('CAT Features (avg over folds)');

    print('CV mean score: {0:.4f}, std: {1:.4f}.'.format(np.mean(scores), np.std(scores)))
    return x_values, prediction

In [ ]:
%%time
import matplotlib.pyplot as plt
%matplotlib inline

x_value_cat, prediction_cat = train_model_cat()

In [ ]:
submission = pd.read_csv('../input/LANL-Earthquake-Prediction/sample_submission.csv')

submission["time_to_failure"] = prediction_cat
submission.to_csv('30_cat.csv', index=False)
submission.head()

# XGB

In [ ]:
num_folds = 5
k_folds = KFold(n_splits=num_folds, shuffle=True, random_state=630)

In [ ]:
def train_model_xgb(X=X_trainset, X_testset=X_testset, y=y_trainset, params=None, k_folds=k_folds, model=None):
    columns = X.columns
    
    x_values = np.zeros(len(X))
    prediction = np.zeros(len(X_testset))
    scores = []
    feature_importance = pd.DataFrame()
    
    for fold_n, (trainset_index, valid_set_index) in enumerate(k_folds.split(X)):
        print('Fold', fold_n, 'started at', time.ctime())
        X_train_per_fold, X_valid_per_fold = X[columns].iloc[trainset_index], X[columns].iloc[valid_set_index]
        y_train_per_fold, y_valid_per_fold = y.iloc[trainset_index], y.iloc[valid_set_index]

        train_data = xgb.DMatrix(data=X_train_per_fold, label=y_train_per_fold, feature_names=columns)
        valid_data = xgb.DMatrix(data=X_valid_per_fold, label=y_valid_per_fold, feature_names=columns)

        watchlist = [(train_data, 'train'), (valid_data, 'valid_data')]
        model = xgb.train(dtrain=train_data, num_boost_round=25000, evals=watchlist, early_stopping_rounds=250, verbose_eval=2500, params=params)
        y_pred_valid = model.predict(xgb.DMatrix(X_valid_per_fold, feature_names=X_trainset.columns), ntree_limit=model.best_ntree_limit)
        y_pred = model.predict(xgb.DMatrix(X_testset, feature_names=X_trainset.columns), ntree_limit=model.best_ntree_limit)

        x_values[valid_set_index] = y_pred_valid.reshape(-1,)
        scores.append(mean_absolute_error(y_valid_per_fold, y_pred_valid))

        prediction += y_pred
       
    prediction /= num_folds
    print('CV mean score: {0:.4f}.'.format(mean_absolute_error(y, x_values)))
    return x_values, prediction

In [ ]:
xgb_params = {'eta': 0.01,
              'max_depth': 6,
              'subsample': 0.8,
              'colsample_bytree': 0.8,
              'colsample_bylevel': 0.8,
              'colsample_bynode': 0.8,
              'lambda': 0.1,
              'alpha' : 0.1,
              'objective': 'reg:linear',
              'eval_metric': 'mae',
              'silent': True,
              'nthread': 4}
x_value_xgb, prediction_xgb = train_model_xgb(params=xgb_params)

In [ ]:
submission = pd.read_csv('../input/LANL-Earthquake-Prediction/sample_submission.csv')

submission["time_to_failure"] = prediction_xgb
submission.to_csv('30_xgb.csv', index=False)
submission.head()